# Reference for ExtendedGravelBedrockEroder tests

This notebook shows the "ground truth" calculations for some of the unit tests used for the `ExtendedGravelBedrockEroder` component.

*(Greg Tucker, Susannah Morey, and Yuval Shmilovitz, March 2025)*

## Aspects to test

### "Instantaneous" tests

Tests should cover the following functionality:

1. Unlimited, homogeneous, fixed width, no attrition

2. Unlimited, homogeneous, fixed width, attrition

3. Unlimited, homogeneous, dynamic width, no attrition

4. Unlimited, multi-lithology, fixed width, attrition

5. Unlimited, multi-size, dynamic width, no attrition

6. Unlimited, multi-size, fixed width, no attrition

7. Unlimited, multi-size, fixed width, attrition

8. Limited, homogeneous, dynamic width, no attrition

9. Limited, multi-size, fixed width, no attrition

10. Limited, multi-size, fixed width, attrition

    


## Basic 1-node instantaneous tests

Here we use a 3x3-node raster grid with a single core node and cell. The cell is 1 x 1 km, and the initial height is 10 m, with one open boundary, for a total slope of 1/100. The runoff rate is 10 m/y, for a discharge out of the cell of 10$^7$ cmy.

### Test 1: unlimited sediment, homogeneous, fixed width, no attrition

Input parameters:

- $k_w = 0.002$ for $Q$ in cmy, and exponent = 1/2
- $D_{50} = 0.01$ m
- Initial sediment thickness 100 m
- $\tau_{*c} = 0.0495$
- $M = 3.97$
- $\rho = 1000$ kg/m$^3$
- $\rho_s = 2650$ kg/m$^3$


In [5]:
import numpy as np
from landlab import RasterModelGrid
#from landlab.components import FlowAccumulator, SoilGrading, GravelBedrockEroder

In [28]:
spy = 3600.0 * 24 * 365.25 # seconds per year

# Functions to use for test calculations
def slope(dz, dx):
    return dz / dx

def discharge(runoff_my, dx):
    area = dx * dx
    return runoff_my * area, runoff_my * area / spy

def fixed_width(kw, m, Q):
    return kw * Q**m

def dynamic_width(rhos, rhow, eps, Q, S, D, tau_star_c, g=9.81):
    R = (rhos - rhow) / rhow
    prefactor = 0.17 * g**(-0.5) * (R * (1 + eps) * tau_staar_c)**(-5/3)
    return prefactor * Q * S**(7/6) / D**1.5

def shear_stress(rho, g, rough, Qcms, w, S):
    return rho * g * rough**0.6 * (Qcms / w)**0.6 * S**0.7

def shields_stress(tau, rho, rhos, g, D):
    return tau / ((rhos - rho) * g * grain_diam)

def sed_flux(intcy, mpm, w, rho, rhos, g, D, tau_star, tau_star_c):
    R = (rhos - rho) / rho
    exshields = max(tau_star - tau_star_c, 0.0)
    Qs_sec = intcy * mpm * w * (R * g)**0.5 * D**1.5 * exshields**1.5
    Qs_yr = Qs_sec * spy
    return Qs_yr

def attrition_thickening_rate(sed_fluxes, beta, dx, cell_area):
    volume_loss_rate_per_length = sed_fluxes * beta
    volume_loss_rate = volume_loss_rate_per_length * dx
    thinning_rate = volume_loss_rate / cell_area
    return -thinning_rate

def outflux_thickening_rate(sed_fluxes, cell_area):
    return -sed_fluxes / cell_area

In [30]:
# Input parameters
sed_init = 100.0
z_init = 10.0
dx = 1000.0
width_coef = 0.002
width_exp = 0.5
runoff = 10.0 # runoff rate, m/y
rho = 1000.0
rhos = 2650.0
rough = 0.05
grain_diam = 0.01
tau_star_c = 0.0495
g = 9.81
mpm = 3.97
intermittency = 0.01

# Predicted outputs
S = slope(z_init, dx)
Q, Qcms = discharge(runoff, dx)
w = fixed_width(width_coef, width_exp, Q)
tau = shear_stress(rho, g, rough, Qcms, w, S)
tau_star = shields_stress(tau, rho, rhos, g, grain_diam)
R = (rhos - rho) / rho # intermediate
Qs = sed_flux(
    intermittency, mpm, w, rho, rhos, g, grain_diam, tau_star, tau_star_c
)
dHdt = outflux_thickening_rate(Qs, dx * dx)
dzdt = dHdt

# Display them
print("PREDICTED VALUES:")
print("slope", S)
print("discharge (cmy)", Q)
print("discharge (cms)", Qcms)
print("channel width (m)", w)
print("bed shear stress (Pa)", tau)
print("Shields stress", tau_star)
print("sediment flux (cmy)", Qs)
print("rate of sediment thickness change (m/y)", dHdt)
print("rate of surface lowering (m/y)", dzdt)

PREDICTED VALUES:
slope 0.01
discharge (cmy) 10000000.0
discharge (cms) 0.3168808781402895
channel width (m) 6.324555320336759
bed shear stress (Pa) 10.739167899631411
Shields stress 0.06634644858141915
sediment flux (cmy) 69.70492856142535
rate of sediment thickness change (m/y) -6.970492856142535e-05
rate of surface lowering (m/y) -6.970492856142535e-05


### Test 2: unlimited sediment, homogeneous, fixed width, attrition

Input parameters:

- $k_w = 0.002$ for $Q$ in cmy, and exponent = 1/2
- $D_{50} = 0.01$ m
- Initial sediment thickness 100 m
- $H_0 = 1$ m
- $\tau_{*c} = 0.0495$
- $M = 3.97$
- $\rho = 1000$ kg/m$^3$
- $\rho_s = 2650$ kg/m$^3$
- $\beta = 0.001$ 1/m


In [34]:
# Input parameters
sed_init = 100.0
z_init = 10.0
dx = 1000.0
width_coef = 0.002
width_exp = 0.5
runoff = 10.0 # runoff rate, m/y
rho = 1000.0
rhos = 2650.0
rough = 0.05
grain_diam = 0.01
tau_star_c = 0.0495
g = 9.81
mpm = 3.97
intermittency = 0.01
beta = 0.002

# Predicted outputs
S = slope(z_init, dx)
Q, Qcms = discharge(runoff, dx)
w = fixed_width(width_coef, width_exp, Q)
tau = shear_stress(rho, g, rough, Qcms, w, S)
tau_star = shields_stress(tau, rho, rhos, g, grain_diam)
R = (rhos - rho) / rho # intermediate
Qs = sed_flux(
    intermittency, mpm, w, rho, rhos, g, grain_diam, tau_star, tau_star_c
)
dHdt_outflux = outflux_thickening_rate(Qs, dx * dx)
dHdt_attrition = attrition_thickening_rate(Qs, beta, dx, dx * dx)
dHdt = dHdt_outflux + dHdt_attrition
dzdt = dHdt

# Display them
print("PREDICTED VALUES:")
print("slope", S)
print("discharge (cmy)", Q)
print("discharge (cms)", Qcms)
print("channel width (m)", w)
print("bed shear stress (Pa)", tau)
print("Shields stress", tau_star)
print("sediment flux (cmy)", Qs)
print("attrition thickness change rate (m/y)", dHdt_attrition)
print("rate of sediment thickness change (m/y)", dHdt)
print("rate of surface lowering (m/y)", dzdt)

PREDICTED VALUES:
slope 0.01
discharge (cmy) 10000000.0
discharge (cms) 0.3168808781402895
channel width (m) 6.324555320336759
bed shear stress (Pa) 10.739167899631411
Shields stress 0.06634644858141915
sediment flux (cmy) 69.70492856142535
attrition thickness change rate (m/y) -0.0001394098571228507
rate of sediment thickness change (m/y) -0.00020911478568427604
rate of surface lowering (m/y) -0.00020911478568427604


In [2]:
def test_inst_fixed_width_unlimited_sed():

    pass
    

0.005